### RAG Pipelines - Data Ingestion to Vector DB Pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader , PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [6]:
### Read all the pdfs inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a direectory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)

    # Find all PDF files recursiveley
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            #Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f" Loaded {len(documents)} pages")
        
        except Exception as e:
            print(f" Error : {e}")

    print(f"\nTotal documents loaded : {len(all_documents)}")
    return all_documents

all_pdf_documents = process_all_pdfs("../data")




Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 84 0 (offset 0)
Ignoring wrong pointing object 110 0 (offset 0)
Ignoring wrong pointing object 112 0 (offset 0)
Ignoring wrong pointing object 142 0 (offset 0)
Ignoring wrong pointing object 164 0 (offset 0)
Ignoring wrong pointing object 171 0 (offset 0)
Ignoring wrong pointing object 173 0 (offset 0)
Ignoring wrong pointing object 189 0 (offset 0)
Ignoring wrong pointing object 201 0 (offset 0)
Ignoring wrong pointing object 207 0 (offset 0)
Ignoring wrong pointing object 260 0 (offset 0)


Found 2 PDF files to process

Processing: NLP_101.pdf
 Loaded 39 pages

Processing: notes.pdf
 Loaded 69 pages

Total documents loaded : 108


In [7]:
all_pdf_documents

[Document(metadata={'producer': 'macOS Version 13.2.1 (Build 22D68) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230331193131Z00'00'", 'moddate': "D:20230331193131Z00'00'", 'source': '..\\data\\pdf\\NLP_101.pdf', 'total_pages': 39, 'page': 0, 'page_label': '1', 'source_file': 'NLP_101.pdf', 'file_type': 'pdf'}, page_content='NATURAL LANGUAGE PROCESSING 101Sarah Rodenbeck, Lead Research Data ScientistRosen Center for Advanced Computing\n1'),
 Document(metadata={'producer': 'macOS Version 13.2.1 (Build 22D68) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230331193131Z00'00'", 'moddate': "D:20230331193131Z00'00'", 'source': '..\\data\\pdf\\NLP_101.pdf', 'total_pages': 39, 'page': 1, 'page_label': '2', 'source_file': 'NLP_101.pdf', 'file_type': 'pdf'}, page_content='Agenda•NLPPrimer•Mechanics & Evolution of NLP•Getting started with NLP\n2'),
 Document(metadata={'producer': 'macOS Version 13.2.1 (Build 22D68) Quartz PDFContext', 'creator': 'PyPDF', 'creationda

In [12]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=500,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
        length_function = len,
        separators=["\n\n","\n"," ",""]
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    #show example of chunk
    if split_docs:
        print(f"\nExample chunk :")
        print(f"Content : {split_docs[0].page_content[:200]}...")
        print(f"Metadata :  {split_docs[0].metadata}")

    return split_docs

In [13]:
chunks = split_documents(all_pdf_documents)
chunks

Split 108 documents into 337 chunks

Example chunk :
Content : NATURAL LANGUAGE PROCESSING 101Sarah Rodenbeck, Lead Research Data ScientistRosen Center for Advanced Computing
1...
Metadata :  {'producer': 'macOS Version 13.2.1 (Build 22D68) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230331193131Z00'00'", 'moddate': "D:20230331193131Z00'00'", 'source': '..\\data\\pdf\\NLP_101.pdf', 'total_pages': 39, 'page': 0, 'page_label': '1', 'source_file': 'NLP_101.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'macOS Version 13.2.1 (Build 22D68) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230331193131Z00'00'", 'moddate': "D:20230331193131Z00'00'", 'source': '..\\data\\pdf\\NLP_101.pdf', 'total_pages': 39, 'page': 0, 'page_label': '1', 'source_file': 'NLP_101.pdf', 'file_type': 'pdf'}, page_content='NATURAL LANGUAGE PROCESSING 101Sarah Rodenbeck, Lead Research Data ScientistRosen Center for Advanced Computing\n1'),
 Document(metadata={'producer': 'macOS Version 13.2.1 (Build 22D68) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230331193131Z00'00'", 'moddate': "D:20230331193131Z00'00'", 'source': '..\\data\\pdf\\NLP_101.pdf', 'total_pages': 39, 'page': 1, 'page_label': '2', 'source_file': 'NLP_101.pdf', 'file_type': 'pdf'}, page_content='Agenda•NLPPrimer•Mechanics & Evolution of NLP•Getting started with NLP\n2'),
 Document(metadata={'producer': 'macOS Version 13.2.1 (Build 22D68) Quartz PDFContext', 'creator': 'PyPDF', 'creationda

### Embedding and Vector DB

In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List , Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

c:\RAG Tutorial\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class EmbeddingManager:
    """Handles document embedding generation using Sentence Transformer"""

    def __init__(self,model_name:str="all-MiniLM-L6-v2"):
        """
            Initialize the embedding manager

            Args:
               model_name :Hugging face model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the Sentence Transformer Model"""
        try:
            print(f"Loading embedding model : {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully . Embedding dimension : {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name} : {e}")
            raise

    def generate_embeddings(self,texts:List[str]) -> np.ndarray:
        """
            Generate embeddings for a list of texts

            Args : texts : List of txt strings to embed
            Returns : numpy arrays of embeddings with shape (len(texsts),embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} textss....")
        embeddings = self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape : {embeddings.shape}")
        return embeddings
    
## initialize the embedding manager
embedding_manager = EmbeddingManager()
embedding_manager
                                                          

Loading embedding model : all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1004.42it/s, Materializing param=pooler.dense.weight]                            
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully . Embedding dimension : 384


### Vector DB

In [ ]:
class VectorStore:
    """Manages document embeddings in a chromadb vector database"""

    def __init__(self,collection_name:str = "pdf_documents",persist_directory:str = "..data/vector_ddb"):
        """
            Initialize Vector DB

            Args : collection_name => Name of the ChromaDB collection
            persis_directory : Dirtectory to persist the vector DB
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_db()

    
    def _initialize_db(self):
        """ Initialize ChromaDB client and collection """
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory,exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata = {"description" : "PDF docuemt embeddings for RAG"}
            )
            print(f"Vector DB initialized . Collection : {self.collection_name}")
            print(f"Existing documents in collection : {self.collection.count()}")
        except Exception as e:
            print(f"Error initializing vector db : {e}")
            raise

    
    def add_documents(self,documents:List[any],embeddings :np.ndarray):
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector db")

        # Prepare datas for chromaDB
        ids = []
        metadata = []
        documents_text = []
        embeddings_list = []

        for i,(doc,embedding) in enumerate(zip(documents,embeddings)):
            #Generate unique ID
            doc_id = f"doc{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            #Prepare Matadata
            metadata = dict(doc.metadata)
            metadata ['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)

            # Document content
            documents_text.append(doc.page_content)

            # Embedding
            embeddings_list.append(embedding.tolist())






